In [1]:
import pandas as pd
import numpy as np
import glob
import os
import sys
from pathlib import Path
from tqdm import tqdm

file_list_g = glob.glob('./../Generated/**.csv')

ip_list = []
for f in file_list_g:
    _ip = f.split('/')[-1].split('_')[-2]
    ip_list.append(_ip)

import time
import datetime
s = "04/11/2016"
START_TIME = time.mktime(datetime.datetime.strptime(s, "%d/%m/%Y").timetuple())
print(START_TIME)

VALID_COLS = [ 'td', 'sa', 'da', 'sp', 'dp', 'pr', 'pkt', 'byt', 'teDelta']

def process(df):
    global START_TIME
    prev = 0
    TS = []
    for i,row in df.iterrows():
        _t = row['teDelta']
        TS.append(_t + prev)
        prev += _t
        
    df['TS']=TS
    df['TS'] = df['TS'] + START_TIME
    df['TS'] = pd.to_datetime(df['TS'], unit='s')
    return df


real_dict = {}
gen_dict = {}

for ip in ip_list:
    # find file
    file_g = glob.glob('./../Generated/**{}**.csv'.format(ip))[0]
    file_r = glob.glob('./../Real/**{}**.csv'.format(ip))[0]
    df_g = pd.read_csv(file_g,index_col=None,usecols=VALID_COLS)
    df_r = pd.read_csv(file_r,index_col=None,usecols=VALID_COLS)
    real_dict[ip] = process(df_r)
    gen_dict[ip] = process(df_g)

# Save the files
path = Path('./../Processed/')
path.mkdir( parents=True, exist_ok=True )

path_r = Path('./../Processed/'+ 'real')
path_r.mkdir( parents=True, exist_ok=True )
path_g = Path('./../Processed/'+ 'generated')
path_g.mkdir( parents=True, exist_ok=True )

for ip in ip_list:
    f_path = ip + '.csv'
    gen_dict[ip].to_csv( os.path.join(path_g, f_path),index=False )

for ip in ip_list:
    f_path = ip + '.csv'
    real_dict[ip].to_csv( os.path.join(path_r, f_path),index=False )

